In [6]:
import os
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from keras.applications import vgg16
from keras.applications import mobilenet
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

%matplotlib inline
cwd = os.getcwd() + '\\'
print(cwd)


c:\projects\notebook\3_classification\


In [7]:
train_path  = cwd + 'data/train'
valid_path  = cwd + 'data/valid'
test_path  = cwd + 'data/test'

In [15]:
train_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    train_path, target_size=(224,224), batch_size=30)
valid_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    valid_path, target_size=(224,224), batch_size=30)
test_batches = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input).flow_from_directory(
    test_path, target_size=(224,224), batch_size=30)

print(len(train_batches))

Found 202 images belonging to 2 classes.
Found 103 images belonging to 2 classes.
Found 451 images belonging to 2 classes.
7


In [9]:
base_model = vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = (224,224, 3))
base_model.summary()

58889256/58889256 [==============================] - 5s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [10]:
for layer in base_model.layers: # lock trainable of base model layers
    layer.trainable = False

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model


last_layer = base_model.get_layer('block5_pool')    # use “get_layer” method to save the last layer of the network
last_output = last_layer.output                     # save the output of the last layer to be the input of the next layer

x = Flatten()(last_output)

x = Dense(64, activation='relu', name='FC_2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax', name='softmax')(x)   # add softmax layer with 2 hidden units

new_model = Model(inputs=base_model.input, outputs=x)
new_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [13]:
new_model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
new_model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)  # steps_per_epoch = train_data_size // batch_size of data_generator

Epoch 1/20


C:\Users\MAC\AppData\Local\Temp\ipykernel_5192\3271892176.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  new_model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=2, epochs=20, verbose=2)  # steps_per_epoch = train_data_size // batch_size of data_generator


4/4 - 1s - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1216 - val_accuracy: 0.9167 - 746ms/epoch - 187ms/step
Epoch 2/20
4/4 - 1s - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.1398 - val_accuracy: 0.9333 - 533ms/epoch - 133ms/step
Epoch 3/20
4/4 - 1s - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.1819 - val_accuracy: 0.9000 - 533ms/epoch - 133ms/step
Epoch 4/20
4/4 - 1s - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0973 - val_accuracy: 0.9500 - 512ms/epoch - 128ms/step
Epoch 5/20
4/4 - 1s - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.1201 - val_accuracy: 0.9167 - 528ms/epoch - 132ms/step
Epoch 6/20
4/4 - 1s - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1751 - val_accuracy: 0.9000 - 520ms/epoch - 130ms/step
Epoch 7/20
4/4 - 1s - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.1607 - val_accuracy: 0.9000 - 518ms/epoch - 130ms/step
Epoch 8/20
4/4 - 1s - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1015 - val_accuracy: 0.9500 - 519ms/epoch - 130ms/step
Epoch 9/20
4/4 - 1s - loss:

In [17]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np

def load_dataset(path):
    data = load_files(path)
    paths = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']))
    return paths, targets

In [18]:
test_files, test_targets = load_dataset(cwd + 'data/test')

In [21]:
from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from tqdm import tqdm

def path_to_tensor(img_path):
    img = load_img(img_path, target_size=(224, 224))    # loads RGB image as PIL.Image.Image type
    x = img_to_array(img)                               # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3) 
    return np.expand_dims(x, axis=0)                    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

test_tensors = preprocess_input(paths_to_tensor(test_files))










100%|██████████| 451/451 [00:01<00:00, 382.85it/s]


In [22]:
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*new_model.evaluate(test_tensors, test_targets)))

15/15 [==============================] - 3s 96ms/step - loss: 0.1068 - accuracy: 0.9601

Testing loss: 0.1068
Testing accuracy: 0.9601


In [23]:
score = new_model.evaluate(test_tensors, test_targets)
print('\n', 'Test accuracy:', score[1])

15/15 [==============================] - 1s 67ms/step - loss: 0.1068 - accuracy: 0.9601

 Test accuracy: 0.9600886702537537
